In [ ]:
%%capture
%run full_setup.py

In [ ]:
from tfidf_corpus_dictionary import get_tfidf_tokendocs_corpus_dict
from gensim.models import LdaModel, LsiModel, CoherenceModel
from sklearn.decomposition import NMF, PCA
from sklearn.random_projection import GaussianRandomProjection, SparseRandomProjection
import numpy as np
from scipy import sparse
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
tfidf_matrix, feature_names, tokenized_docs, corpus, dictionary = get_tfidf_tokendocs_corpus_dict(df, max_df=0.5, min_df=5, max_features=1000)

In [ ]:
from coherence_by_topics import coherence_by_topics
from coherence_by_words import coherence_by_words

In [ ]:
topics = [5, 10, 20, 50]

In [ ]:
evaluation_by_topics = {}

for n_topics in topics:
    metrics_words = coherence_by_topics(n = n_topics, corpus=corpus, dictionary=dictionary,
                               texts=tokenized_docs, feature_names=feature_names, tfidf=tfidf_matrix)
    
    evaluation_by_topics[n_topics] = metrics_words

In [ ]:
evaluation_by_topics[5]

In [ ]:
evaluation_by_topics[10]

In [ ]:
evaluation_by_topics[20]

In [ ]:
evaluation_by_topics[50]

In [ ]:
words = [10, 100, 1000, 10000]

In [ ]:
evaluation_by_words = {}

for n_words in words:
    metrics_words = coherence_by_words(df, n = n_words)
    evaluation_by_words[n_words] = metrics_words

In [ ]:
evaluation_by_words[10]

In [ ]:
evaluation_by_words[100]

In [ ]:
evaluation_by_words[1000]

In [ ]:
evaluation_by_words[10000]

In [ ]:
from tables import tables

In [ ]:
tables(evaluation_by_topics, 'topics')

In [ ]:
tables(evaluation_by_words, 'words')

In [ ]:
from plots import plots

In [ ]:
plots(evaluation_by_topics, 'topics')

In [ ]:
plots(evaluation_by_words, 'words')

In [ ]:
from display_topics import display_topics

Now we'll fit the LDA model with the number of topics that yields the highest coherence

In [ ]:
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=5,
                     alpha='symmetric', eta='auto', passes=5, random_state=1)

In [ ]:
display_topics('LDA', lda_model, feature_names)

Now we're going to do the same for LSA

In [ ]:
lsi_model = LsiModel(corpus, id2word=dictionary, num_topics=5, random_seed = 1)

In [ ]:
display_topics('LSA', lsi_model, feature_names)

In [ ]:
nmf_model = NMF(n_components=5, random_state=1).fit(tfidf_matrix)

In [ ]:
display_topics('NMF', nmf_model, feature_names)

In [ ]:
tfidf_matrix_dense = tfidf_matrix.todense() if sparse.issparse(tfidf_matrix) else tfidf_matrix

# Convert to numpy array
tfidf_matrix_array = np.asarray(tfidf_matrix_dense)

# Centering
mean_tfidf = np.mean(tfidf_matrix_array, axis=0)  # Calculate the mean of each column
centered_tfidf_matrix = tfidf_matrix_array - mean_tfidf

pca_model = PCA(n_components=5, random_state=1).fit(centered_tfidf_matrix)

In [ ]:
display_topics('PCA', pca_model, feature_names)

In [ ]:
rp_model = GaussianRandomProjection(n_components=5, random_state=1).fit(tfidf_matrix)

In [ ]:
display_topics('RP', rp_model, feature_names)